# Laboratorio #6: Apriori
___
Marcelo Delgado Mora C12510

Felipe Mena Rodríguez C24721

Jose Mora Monge C15114

___

##Imports Necesarios


In [1]:
import pandas as pd
from itertools import combinations, chain
from google.colab import files
import time

## Dataset

In [2]:
# Montar drive:
archivo = files.upload()

Saving retail_data.csv to retail_data.csv


In [3]:
# Obtener dataset:
transactions = pd.read_csv('retail_data.csv')
transactions.head()

,Trans_ID,Product1,Product2,Product3
0,1,Bread,Butter,Dairy
1,2,Nachos,Butter,Dairy
2,3,Juice,Jam,Egg
3,4,Juice,Jam,Egg
4,5,Juice,Vegetable,Salad


In [4]:
len(transactions)

2000

## Limpieza y Preparación de Datos

In [5]:
# Eliminar columnas con valores nulos:
transactions = transactions.dropna()
len(transactions)

2000

In [6]:
transactions.drop(['Trans_ID'], axis=1, inplace=True)
transactions.head()

,Product1,Product2,Product3
0,Bread,Butter,Dairy
1,Nachos,Butter,Dairy
2,Juice,Jam,Egg
3,Juice,Jam,Egg
4,Juice,Vegetable,Salad


In [7]:
transaction_list = transactions.apply(lambda row: [item for item in row if pd.notnull(item)], axis=1).tolist()
transaction_list = [set(t) for t in transaction_list]
print(transaction_list)

[{'Dairy', 'Bread', 'Butter'}, {'Dairy', 'Nachos', 'Butter'}, {'Jam', 'Egg', 'Juice'}, {'Jam', 'Egg', 'Juice'}, {'Salad', 'Vegetable', 'Juice'}, {'Egg', 'Bread', 'Butter'}, {'Dairy', 'Salsa', 'Nachos'}, {'Jam', 'Egg', 'Bread'}, {'Jam', 'Egg', 'Juice'}, {'Salad', 'Vegetable', 'Fruits'}, {'Egg', 'Bread', 'Butter'}, {'Egg', 'Bread', 'Butter'}, {'Dairy', 'Salsa', 'Nachos'}, {'Jam', 'Egg', 'Juice'}, {'Jam', 'Egg', 'Juice'}, {'Salsa', 'Nachos', 'Egg'}, {'Dairy', 'Salsa', 'Nachos'}, {'Egg', 'Bread', 'Butter'}, {'Jam', 'Dairy', 'Bread'}, {'Egg', 'Bread', 'Butter'}, {'Jam', 'Egg', 'Juice'}, {'Egg', 'Bread', 'Butter'}, {'Jam', 'Dairy', 'Bread'}, {'Jam', 'Egg', 'Juice'}, {'Jam', 'Egg', 'Juice'}, {'Salad', 'Vegetable', 'Juice'}, {'Jam', 'Egg', 'Juice'}, {'Jam', 'Dairy', 'Bread'}, {'Salad', 'Vegetable', 'Juice'}, {'Jam', 'Egg', 'Juice'}, {'Egg', 'Bread', 'Butter'}, {'Jam', 'Egg', 'Juice'}, {'Salad', 'Vegetable', 'Fruits'}, {'Egg', 'Bread', 'Butter'}, {'Egg', 'Bread', 'Butter'}, {'Jam', 'Dairy', 'Br

# Apoyo y Confianza

* Para este laboratorio, se escoge un apoyo de 0.1 debido a que es un dataset pequeño, y una confianza de 0.5 por la misma razón, para generar resultados interesantes sin descartar relaciones pequeñas.

## Algoritmo Apriori

In [8]:
# Obtener itemset podado:
def prune_itemset(item_counts, min_support):
  itemset= []
  for item in item_counts:
    if item_counts[item] >= min_support:
      itemset.append(item)
  return itemset

# Generar las combinaciones de n elementos:
def generate_combinations(items, n):
  unique = set().union(*items)
  return list(combinations(unique, n))

# Generar número de ocurrencias para las combinaciones
def get_combinations_counts(item_combinations):
  counts = {}
  for combinations in item_combinations:
    frequency = 0
    for transaction in transaction_list:
      if set(combinations).issubset(transaction):
        frequency += 1
    counts[combinations] = frequency / len(transactions)
  return counts

In [9]:
def scratch_apriori (transaction_list, min_support):
  all_frequent_itemsets = []
  candidates = generate_combinations(transaction_list, 1)
  supports = get_combinations_counts(candidates)
  frequent_k_itemsets = prune_itemset(supports, min_support)
  while frequent_k_itemsets:
    for itemset in frequent_k_itemsets:
      all_frequent_itemsets.append(itemset)
    candidates = generate_combinations(frequent_k_itemsets, len(frequent_k_itemsets[0]) + 1)
    supports = get_combinations_counts(candidates)
    frequent_k_itemsets = prune_itemset(supports, min_support)

  return get_combinations_counts(all_frequent_itemsets)

def generate_rules(frequent_item_supports, min_confidence):
    canon = lambda it: tuple(sorted(it))

    freq = {canon(items): supp
            for items, supp in frequent_item_supports.items()}

    rules = []

    for itemset, supp_itemset in freq.items():
        k = len(itemset)
        if k < 2:
            continue

        for i in range(1, k):
            for antecedent in combinations(itemset, i):
                A = canon(antecedent)
                B = canon(set(itemset) - set(A))

                supp_A = freq[A]
                conf   = supp_itemset / supp_A

                if conf >= min_confidence:
                    rules.append({
                        'antecedents': A,
                        'consequents': B,
                        'support': supp_itemset,
                        'confidence': conf
                    })

    return rules


In [10]:
tiempo_scratch_ini = time.perf_counter()
frequent_itemsets = scratch_apriori(transaction_list, 0.1)
rules = generate_rules(frequent_itemsets, 0.5)
tiempo_scratch_fin = time.perf_counter() - tiempo_scratch_ini

df_itemsets = (
    pd.DataFrame(
        [{"itemsets": set(itemset), "support": support}
         for itemset, support in frequent_itemsets.items()])
      .sort_values(by='support', ascending=False)
      .reset_index(drop=True)
)

df_itemsets['itemset_disp'] = (
    df_itemsets['itemsets'].apply(lambda s: ', '.join(sorted(s)))
)

print("\n----------------Frequent itemsets-------------------\n")
print(df_itemsets[['itemset_disp', 'support']].to_string(index=False))

df_rules = (
    pd.DataFrame(rules)
      .assign(
          antecedents=lambda d: d['antecedents'].apply(lambda s: ', '.join(sorted(s))),
          consequents=lambda d: d['consequents'].apply(lambda s: ', '.join(sorted(s))))
      .sort_values(by='confidence', ascending=False)
      .reset_index(drop=True)
)

print("\n-------------Association rules----------------------\n")
print(df_rules.to_string(index=False))


----------------Frequent itemsets-------------------

            itemset_disp  support
                     Egg   0.5760
                   Bread   0.3510
                     Jam   0.3410
                Egg, Jam   0.2570
              Bread, Egg   0.2505
                   Juice   0.2465
                  Butter   0.2465
                   Salad   0.2210
                  Nachos   0.2200
               Vegetable   0.2160
           Bread, Butter   0.2145
        Salad, Vegetable   0.2090
              Egg, Juice   0.2080
                   Dairy   0.2030
         Egg, Jam, Juice   0.2010
              Jam, Juice   0.2010
                   Salsa   0.1965
             Butter, Egg   0.1945
      Bread, Butter, Egg   0.1945
           Nachos, Salsa   0.1880
                  Fruits   0.1825
           Fruits, Salad   0.1825
       Fruits, Vegetable   0.1705
Fruits, Salad, Vegetable   0.1705
              Bread, Jam   0.1280
              Egg, Salsa   0.1175
      Egg, Nachos, Salsa   

## **(Extra)** Formas de mejorar el Algoritmo Apriori Básico

* El Algoritmo Apriori en sì no es muy eficiente por su naturaleza, debido a que debe de revisar constantemente el dataset original. Algunas mejoras que se pueden implementar al algoritmo son:

  1. Usar Hash Tables para revisar con frecuencia y rapidez los conjuntos de elementos candidatos, y así descartar los no deseados de manera más rápida.
  2. Dividir el dataset en particiones, para realizar operaciones más rápidas, y luego revisar en conjunto los resultados.
  3. Reducir el tamaño del dataset una vez que se detectan transacciones sin ningún conjunto frecuente, para reducir el conjunto de datos a revisar.
  4. Utilizar técnicas de recuento de conjuntos concurrentes, para distribuir y reducir la carga de trabajo.
  5. Utilizar variaciones/derivados del algoritmo Apriori que siguen el mismo concepto principal pero que realizan cambios u optimizaciones para reducir su coste computacional.

## Apriori Avanzado


In [11]:
transactions['product_list'] = (
    transactions[['Product1', 'Product2', 'Product3']]
    .apply(lambda row: row.dropna().tolist(), axis=1)
)

print(transactions[['product_list']].head())

                product_list
0     [Bread, Butter, Dairy]
1    [Nachos, Butter, Dairy]
2          [Juice, Jam, Egg]
3          [Juice, Jam, Egg]
4  [Juice, Vegetable, Salad]


In [13]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

tiempo_mlxtend_ini = time.perf_counter()
te = TransactionEncoder()
te_ary = te.fit_transform(transactions['product_list'].tolist())
encoded_transactions = pd.DataFrame(te_ary, columns=te.columns_, dtype=bool)

freq_apriori = apriori(encoded_transactions, min_support=0.1, use_colnames=True)
asso_apriori = association_rules(freq_apriori, metric='confidence', min_threshold=0.5)
tiempo_mlxtend_fin = time.perf_counter() - tiempo_mlxtend_ini

frequent_itemsets = (
      freq_apriori
      .sort_values(by='support', ascending=False)
      .reset_index(drop=True)
)

rules = (
      asso_apriori
      .sort_values(by='confidence', ascending=False)
      .reset_index(drop=True)
)

frequent_itemsets['itemset_disp'] = (
    frequent_itemsets['itemsets'].apply(lambda s: ', '.join(sorted(s)))
)
rules['antecedents'] = rules['antecedents'].apply(lambda s: ', '.join(sorted(s)))
rules['consequents'] = rules['consequents'].apply(lambda s: ', '.join(sorted(s)))

print("\n-------------------Frequent itemsets----------------\n")
print(frequent_itemsets[['itemset_disp', 'support']].to_string(index=False))

print("\n----------------Association rules---------------\n")
print(rules[['antecedents', 'consequents', 'support', 'confidence']].to_string(index=False))


-------------------Frequent itemsets----------------

            itemset_disp  support
                     Egg   0.5760
                   Bread   0.3510
                     Jam   0.3410
                Egg, Jam   0.2570
              Bread, Egg   0.2505
                   Juice   0.2465
                  Butter   0.2465
                   Salad   0.2210
                  Nachos   0.2200
               Vegetable   0.2160
           Bread, Butter   0.2145
        Salad, Vegetable   0.2090
              Egg, Juice   0.2080
                   Dairy   0.2030
         Egg, Jam, Juice   0.2010
              Jam, Juice   0.2010
                   Salsa   0.1965
      Bread, Butter, Egg   0.1945
             Butter, Egg   0.1945
           Nachos, Salsa   0.1880
           Fruits, Salad   0.1825
                  Fruits   0.1825
Fruits, Salad, Vegetable   0.1705
       Fruits, Vegetable   0.1705
              Bread, Jam   0.1280
             Egg, Nachos   0.1175
      Egg, Nachos, Salsa   

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Reporte Y Contraste de Algoritmos

In [14]:
same_itemsets = (
    set(map(frozenset, df_itemsets['itemsets'])) ==
    set(map(frozenset, frequent_itemsets['itemsets']))
)

same_rules = (
    {(frozenset(a), frozenset(c))
     for a, c in zip(df_rules['antecedents'],
                     df_rules['consequents'])}
    ==
    {(frozenset(a), frozenset(c))
     for a, c in zip(rules['antecedents'],
                     rules['consequents'])}
)

print("¿Itemsets idénticos? ", same_itemsets,
      "\n¿Reglas idénticas?  ", same_rules)



¿Itemsets idénticos?  True 
¿Reglas idénticas?   True


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

### Conjuntos Finales y Reglas de Asociación

* Los Itemsets Frecuentes y las Reglas de Asociación son idénticas entre Algoritmos, con la única diferencia en el orden en que se generan y reportan en el Dataframe, cuando hay 2 items con el mismo valor a comparar/filtrar.

In [15]:
print("Tiempo ejecución Algoritmo Apriori Básico: ", tiempo_scratch_fin, " segundos")
print("Tiempo ejecución Algoritmo Apriori Avanzado: ", tiempo_mlxtend_fin, " segundos")

Tiempo ejecución Algoritmo Apriori Básico:  0.17084343500000898  segundos
Tiempo ejecución Algoritmo Apriori Avanzado:  0.01612818099999913  segundos


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

### Tiempos de ejecución

* El tiempo de ejecución es más rápido para el Algoritmo Apriori Avanzado, implementado utilizando la librería mlxtend, que el Algoritmo Apriori Básico implementado a mano.

### Conclusión

* La diferencia entre implementaciones es mínima, ambas producen los mismos resultados y la diferencia de tiempo es en milisegundos, pero debido a la facilidad de uso y optimizaciones implementadas por mlxtend, es mejor utilizar esta opción.